# Example 7
Un ladrón intenta decidir que cosas llevarse tras haber conseguido entrar a una tienda de electrodomésticos. El problema es que debe elegir entre 10 electrodomésticos, todos de distinto precio y peso, pero solo trajo 2 bolsas para llevarlos, con 51 y 78 kilogramos de capacidad. Como son productos que están en exposición, solo hay uno de cada tipo y, por lo tanto, no podría llevarse dos o más productos iguales.
Describa un método que le ayude a este malhechor a elegir la combinación óptima de productos (o una que esté lo más cercano posible a la óptima), de forma de obtener la mayor ganancia y no excederse de la capacidad de sus bolsas.
Detalle y justi que todas las decisiones tomadas con respecto al método, parámetros empleados, representación, etc. 

Implemente el método propuesto y úselo para encontrar e informar la mejor solución que pueda obtener.
Para asegurarse de haber obtenido resultados consistentes, realice 10 ejecuciones independientes e informe el mejor resultado encontrado en cada ejecución junto con su evaluación y la cantidad de generaciones que se necesitaron para obtenerlo.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
import mpld3

%matplotlib inline
mpld3.enable_notebook()

from utils.busqueda_local import hill_climb
from deap import base, creator, tools, algorithms

In [2]:
# -*- coding: utf-8 -*-
"""
Example 7: Ayudando al ladrón
"""
from numpy import array
import random

# El peso en kilogramos de cada uno de los productos
pesos   = array([ 9.0, 4.5, 11.5, 10.0, 29.5, 30.5, 35.0, 37.5, 38.0, 15.0 ])

# El precio en pesos de cada uno de los productos
precios = array([ 780, 350,  890,  360,  940,  750,  740,  790,  800,  160 ])

# Las capacidades en kilogramos de cada una de las bolsas
bolsas  = array([ 51, 78 ])

MAX_ITEM = 10
MAX_WEIGHT = [bolsas[0], bolsas[1]]

### Build Dictionary

In [3]:
items = {}
for i in range(pesos.shape[0]):
    items[i] = (pesos[i],precios[i])
items

{0: (9.0, 780),
 1: (4.5, 350),
 2: (11.5, 890),
 3: (10.0, 360),
 4: (29.5, 940),
 5: (30.5, 750),
 6: (35.0, 740),
 7: (37.5, 790),
 8: (38.0, 800),
 9: (15.0, 160)}

# Para N mochilas

In [4]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", list, fitness=creator.Fitness)

In [5]:
def crearMochila():
    return {random.randrange(MAX_ITEM)}    

In [6]:
toolbox = base.Toolbox()
toolbox.register("individual", tools.initRepeat, creator.Individual, crearMochila, len(bolsas))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [7]:
import collections

def evaluar1Mochila(individual):
    peso = 0.0
    valor = 0.0
    for item in individual:
        valor += items[item][1]
        peso += items[item][0]
    if len(individual) > MAX_ITEM or peso > MAX_WEIGHT[0]:
        return 10000, -10000             
    return peso, valor

def appendAll(lista):
    b = []
    for asd in lista:
        for j in list(asd):
            b.append(j)
    return b

def intersection(lista):
    b = appendAll(lista)
    return [item for item, count in collections.Counter(b).items() if count > 1]

def evalNKnapsack(individual):
    peso, valor = 0.0, 0.0
    for i in range(len(bolsas)):
        for item in individual[i]:
            valor += items[item][1]
            peso += items[item][0]
        if len(individual[i]) > MAX_ITEM or peso > MAX_WEIGHT[i]:
            return 10000, -10000             # Asegurar que no haya sobrepeso
    
    if (ind != {} for ind in individual):
        if intersection(individual) != []:
            return 10000, -100000  # Asegurar items no repetidos
    elif (len(appendAll(individual)) == 0):
            return 10000, -100000  # No me queiro ir con las mochilas vacias
    return peso, valor

In [8]:
a = [{0,2,3},{1},{4}]
evalNKnapsack(a)

(35.0, 2380.0)

In [9]:
def mutSetNBags(individual, intpb=0.5):
    """Mutation that pops or add an element."""
    for bolsa in range(len(bolsas)):
        if random.random() < intpb:
            if len(individual[bolsa]) > 0:     
                individual[bolsa].remove(random.choice(sorted(tuple(individual[bolsa]))))
        else:
            individual[bolsa].add(random.randrange(MAX_ITEM))
    return individual,

In [10]:
def cxSetNBags(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    for bolsa in range(len(bolsas)):
        temp = set(ind1[bolsa])                # Used in order to keep type
        ind1[bolsa] &= ind2[bolsa]             # Intersection (inplace)
        ind2[bolsa] ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

In [11]:
toolbox.register("evaluate", evalNKnapsack)
toolbox.register("mate", cxSetNBags)
toolbox.register("mutate", mutSetNBags)
toolbox.register("select", tools.selNSGA2)

In [12]:
def main():
    GENERACIONES = 20
    POBLACION = 10
    LAMBDA = 80
    CXPB = 0.7
    MUTPB = 0.3
    
    
    # Save History
    history = tools.History()
    toolbox.decorate("mate", history.decorator)
    toolbox.decorate("mutate", history.decorator)
    
    pop = toolbox.population(n=POBLACION)
    history.update(pop)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("max", np.max, axis=0)    
    algorithms.eaMuPlusLambda(pop, toolbox, POBLACION, LAMBDA, CXPB, MUTPB, GENERACIONES, stats,
                              halloffame=hof)
    
    return history, pop, stats, hof

In [13]:
history, pop, log, hof = main()
bestCombination = hof[len(hof)-1]
print "Genealogy: ", history.getGenealogy(bestCombination, 50)
print "\nMEJOR:"
for mochila in range(len(bolsas)):
    print "\nMochila: ", bestCombination[mochila], "- Peso y Valor: ", evaluar1Mochila(bestCombination[mochila])

gen	nevals	avg                	max            
0  	10    	[  2035.5 -18897. ]	[10000.  1720.]
1  	80    	[ 13.65 720.  ]    	[  61. 2780.]  
2  	80    	[  21.2 1069. ]    	[  61. 2780.]  
3  	80    	[  21.9 1115. ]    	[  61. 2780.]  
4  	80    	[  33.2 1543. ]    	[  76. 2940.]  
5  	80    	[  32.3 1736. ]    	[  76. 2940.]  
6  	80    	[  34.75 1847.  ]  	[  76. 2940.]  
7  	80    	[  32.35 1815.  ]  	[  65.5 3130. ]
8  	80    	[  28.95 1686.  ]  	[  65.5 3130. ]
9  	80    	[  27.05 1572.  ]  	[  65.5 3130. ]
10 	80    	[  29.55 1774.  ]  	[  65.5 3130. ]
11 	80    	[  30.25 1820.  ]  	[  65.5 3130. ]
12 	80    	[  30.7 1855. ]    	[  65.5 3130. ]
13 	80    	[  35.65 2052.  ]  	[  65.5 3130. ]
14 	80    	[  27.05 1572.  ]  	[  65.5 3130. ]
15 	80    	[  32.4 1775. ]    	[  72.5 3170. ]
16 	80    	[  34.75 1924.  ]  	[  72.5 3170. ]
17 	80    	[  31.6 1883. ]    	[  72.5 3170. ]
18 	80    	[  29.8 1735. ]    	[  72.5 3170. ]
19 	80    	[  26.75 1532.  ]  	[  72.5 3170. ]
20 	80    	[ 